In [22]:
import pandas as pd
import numpy as np
import joblib
import ast
from joblib import Parallel, delayed
import json
import os


In [12]:
item_data = pd.read_csv('/opt/ml/wine/data/wine_df.csv')

ValueError: Must specify a fill 'value' or 'method'.

In [10]:
notes = ['Red Fruit','Tropical','Tree Fruit','Oaky',
        'Ageing','Black Fruit','Citrus','Dried Fruit','Earthy',
        'Floral','Microbio','Spices', 'Vegetal']

def drop_columns(df):
    to_drop = ['Red Fruit','Tropical','Tree Fruit','Oaky',
               'Ageing','Black Fruit','Citrus','Dried Fruit','Earthy',
               'Floral','Microbio','Spices', 'Vegetal',

               'Unnamed: 58', 'None_count', 'Unnamed: 60', 'Unnamed: 61', 
               'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64']
    
    df.drop(to_drop, axis = 1, inplace= True)
    
    return df

In [66]:
def feature_mapper(df, column):
    unique_val = df[column].unique()
    feature2idx = {f:i for i, f in enumerate(unique_val)}
    idx2feature = {i:f for i, f in enumerate(unique_val)}

    if not os.path.exists('/opt/ml/wine/code/meta_data/'): 
        os.makedirs('/opt/ml/wine/code/meta_data/')

    with open(f'/opt/ml/wine/code/feature_map/{column}2idx.json','w',encoding='utf-8') as f:  
        json.dump(feature2idx, f, ensure_ascii=False)
    with open(f'/opt/ml/wine/code/feature_map/idx2{column}.json','w',encoding='utf-8') as f:  
        json.dump(idx2feature, f, ensure_ascii=False)

    return feature2idx, idx2feature

def map_all_single_features(df):
    single_category_columns = ['country', 'region', 'winery', 'winetype', 'vintage', 'house', 'wine_style']
    for c in single_category_columns:
        feature_mapper(df, c)

    return 

def str2list(x):
    if x[0] != '[':
        list = [x]
    else: 
        list = ast.literal_eval(x)

    return list

def multi_feature_mapper(df, column):

    df[column] = df[column].apply(x = lambda x: str2list(str))

    if not os.path.exists('/opt/ml/wine/code/meta_data/'): 
        os.makedirs('/opt/ml/wine/code/meta_data/')

    with open(f'/opt/ml/wine/code/feature_map/{column}2idx.json','w',encoding='utf-8') as f:  
        json.dump(feature2idx, f, ensure_ascii=False)
    with open(f'/opt/ml/wine/code/feature_map/idx2{column}.json','w',encoding='utf-8') as f:  
        json.dump(idx2feature, f, ensure_ascii=False)

    return feature2idx, idx2feature

In [35]:
map_all_single_features(item_data)

In [ ]:
string_columns  = single_category_columns = ['country', 'region', 'winery', 'winetype', 'vintage', 'house', 'wine_style']

In [60]:
string_columns = ['url', 'country', 'region', 'winery', 'winetype',  'name',
                           'house', 'wine_style']
list_columns = ['grape','pairing']

continous_columns = ['price', 'rating', 'num_votes', 
                     'star5', 'star4','star3', 'star2', 'star1', 
                     'Red Fruit_count','Tropical_count', 'Tree Fruit_count',
                     'Oaky_count', 'Ageing_count', 'Black Fruit_count','Citrus_count',
                     'Dried Fruit_count',  'Earthy_count',  'Floral_count', 
                     'Microbio_count',  'Spices_count', 'Vegetal_count']

dict_columns = ['Red_Fruit_child', 'Tree_Fruit_child','Oaky_child',
                'Ageing_child', 'Citrus_child','Tropical_child',
                'Black_Fruit_child', 'Dried_Fruit_child', 'Earthy_child',
                'Floral_child', 'Microbio_child','Spices_child', 'Vegetal_child']


with open(f'/opt/ml/wine/code/data/meta_data/dict_columns.json','w',encoding='utf-8') as f:  
        json.dump(dict_columns , f, ensure_ascii=False)
with open(f'/opt/ml/wine/code/data/meta_data/continous_columns.json','w',encoding='utf-8') as f:  
        json.dump(continous_columns, f, ensure_ascii=False)
with open(f'/opt/ml/wine/code/data/meta_data/list_columns.json','w',encoding='utf-8') as f:  
        json.dump(list_columns, f, ensure_ascii=False)
with open(f'/opt/ml/wine/code/data/meta_data/string_columns.json','w',encoding='utf-8') as f:  
        json.dump(string_columns, f, ensure_ascii=False)

In [63]:
def fill_na(df):
    with open(f'/opt/ml/wine/code/data/meta_data/dict_columns.json','r',encoding='utf-8') as f:  
        dic_c = json.load(f)

    with open(f'/opt/ml/wine/code/data/meta_data/continous_columns.json','r',encoding='utf-8') as f:  
        con_c = json.load(f)

    with open(f'/opt/ml/wine/code/data/meta_data/list_columns.json','r',encoding='utf-8') as f:  
        list_c = json.load(f)

    with open(f'/opt/ml/wine/code/data/meta_data/string_columns.json','r',encoding='utf-8') as f:  
        str_c = json.load(f)

    df[dic_c] = df[dic_c].fillna("{}")
    df[con_c] = df[con_c].fillna(np.nan)
    df[list_c] = df[list_c].fillna("[]")    
    df[str_c] = df[str_c].fillna("")

    return df


In [64]:
item_data = fill_na(item_data)

In [68]:
item_data['grape'] = item_data['grape'].apply(lambda x: str2list(x))
